In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 849kB 4.1MB/s 
     |████████████████████████████████| 184kB 11.3MB/s 
     |████████████████████████████████| 276kB 13.1MB/s 
     |████████████████████████████████| 112kB 13.8MB/s 
     |████████████████████████████████| 829kB 12.0MB/s 
     |████████████████████████████████| 1.3MB 16.4MB/s 
     |████████████████████████████████| 296kB 29.5MB/s 
     |████████████████████████████████| 143kB 25.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=62707f8e83cd7c01f06da4daa910fe4a423060e1f181b79eff2c3a3c59b6a763
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=ed95ddede3f93ac732d7221eb38cdfbc46b5bb16c214514b427ed5569cd646aa
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML

In [ ]:
import torch
import torchvision
import pytorch_lightning as pl

In [ ]:

class MNISTDataModule(pl.LightningDataModule):

    def __init__(self, path = '../data', batch_size = 64):
        super().__init__()
        self.path = path
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.mnist_train = torchvision.datasets.MNIST(
            self.path, train=True, download=True, transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
          )
        self.mnist_val = torchvision.datasets.MNIST(
            self.path, train=False, download=True, transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
          )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.mnist_val, batch_size=self.batch_size)

In [ ]:




class Modelo(pl.LightningModule):

    def __init__(self, config, n_channels = 1, n_outputs = 10):
        super().__init__()
        self.save_hyperparameters(config)
        self.conv1 = block(n_channels, self.hparams.filters1)
        self.conv2 = block(self.hparams.filters1, self.hparams.filters2)  
        self.fc = torch.nn.Linear(self.hparams.filters2*7*7, n_outputs)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('loss', loss)
        self.log('acc', accuracy(y_hat, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', accuracy(y_hat, y), prog_bar=True)

    def configure_optimizers(self):
        return getattr(torch.optim, self.hparams.optimizer)(self.parameters(), lr=self.hparams.lr)

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping #earlystoopinf
from pytorch_lightning.callbacks import ModelCheckpoint


config = {
    'lr': 3e-4,
    'optimizer': 'Adam',
    'batch_size': 64,
    'filters1': 32,
    'filters2': 64
}

modelo= Modelo(congfig)
dm= MNISTDataModule(batch_size =config['batch_size'])

early_stop_callback = EarlyStopping(
   monitor='val_acc', # el valor en el modelo
   patience=3, #cuantas epoch tiene que pasar seguida sin que mejore
   verbose=False,
   mode='max'
)

#### chepoint es para guardar el mejor modelo entrenado
checkpoint = ModelCheckpoint(
    dirpath='./',              #donde queremos que guarde el modelo
    filename='modelo-{val_acc:.5f}', #nombre y ademas las variables que queramos
    save_top_k=1, #el numero de mejores moedlos que guarde
    monitor='val_acc', # la metrica a memorizar
    mode='max'  #el valor mas grande modo max
)                                                                             



trainer = pl.Trainer(
    gpus=1,
    callbacks=[early_stop_callback,checkpoint ]
)

trainer.fit(modelo, dm)


In [ ]:
modelo=Modelo.load_from_checkpoint(checkpoint_path="modelo-val_acc=0.99090.ckpt")
modelo.hparams #y te da todo los hiperparametros declarados